# yfinance: Demonstration notebook (prices, fundamentals, options, holders, ESG, news)

This notebook demonstrates (with robust try/except) most objects you can retrieve via `yfinance.Ticker` and `yf.download()`.

Notes

- Yahoo Finance access is unofficial; some fields may be missing or may change over time.
- Availability depends on the instrument (equity vs ETF vs crypto) and region.
- If you get throttled, reduce request frequency and cache responses.


In [1]:
# If needed (uncomment in a fresh environment):
# !pip -q install yfinance pandas numpy

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

## 1) Helper utilities (safe calls + summarization)

In [2]:
from typing import Any, Callable, List, Tuple, Optional

def summarize_value(val: Any, max_keys: int = 25, max_cols: int = 15) -> str:
    """Compact summary for display/logging."""
    if val is None:
        return "None"
    if isinstance(val, pd.DataFrame):
        cols = list(val.columns)[:max_cols]
        return f"DataFrame shape={val.shape}, cols={cols}"
    if isinstance(val, pd.Series):
        return f"Series len={len(val)}, name={val.name}, dtype={val.dtype}"
    if isinstance(val, dict):
        keys = list(val.keys())[:max_keys]
        return f"dict keys(n={len(val)}): {keys}"
    if isinstance(val, (list, tuple, set)):
        return f"{type(val).__name__} len={len(val)}"
    return f"{type(val).__name__}"

def safe_getattr(obj: Any, attr: str) -> Tuple[bool, Any, Optional[str]]:
    """Get an attribute or property with exception capture."""
    try:
        v = getattr(obj, attr)
        return True, v, None
    except Exception as e:
        return False, None, repr(e)

def safe_call(fn: Callable[[], Any]) -> Tuple[bool, Any, Optional[str]]:
    """Call a zero-arg function with exception capture."""
    try:
        v = fn()
        return True, v, None
    except Exception as e:
        return False, None, repr(e)

def probe_ticker(symbol: str, fields: List[str]) -> pd.DataFrame:
    """Probe many yfinance.Ticker properties and return a diagnostics table."""
    t = yf.Ticker(symbol)
    rows = []
    for f in fields:
        ok, v, err = safe_getattr(t, f)
        rows.append({
            "field": f,
            "ok": ok,
            "type": type(v).__name__ if ok else None,
            "summary": summarize_value(v) if ok else None,
            "error": err if not ok else None
        })
    return (pd.DataFrame(rows)
            .sort_values(["ok", "field"], ascending=[False, True])
            .reset_index(drop=True))

pd.set_option("display.max_colwidth", 140)

## 2) Choose symbols (equity + ETF example)

In [3]:
EQUITY = "AAPL"   # change to any equity ticker
ETF    = "SPY"    # ETF example for funds_data

t_equity = yf.Ticker(EQUITY)
t_etf    = yf.Ticker(ETF)

(EQUITY, ETF)

('AAPL', 'SPY')

## 3) Price history (OHLCV) + embedded corporate actions

In [34]:
# Basic OHLCV history
hist = t_equity.history(period="1y", interval="1d", auto_adjust=False, actions=False)
hist.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-12-20 00:00:00-05:00,248.039993,255.000000,245.690002,254.490005,253.344208,147495300
2024-12-23 00:00:00-05:00,254.770004,255.649994,253.449997,255.270004,254.120682,40858800
2024-12-24 00:00:00-05:00,255.490005,258.209991,255.289993,258.200012,257.037506,23234700
2024-12-26 00:00:00-05:00,258.190002,260.100006,257.630005,259.019989,257.853790,27237100
2024-12-27 00:00:00-05:00,257.829987,258.700012,253.059998,255.589996,254.439240,42355300


In [35]:
# History with embedded actions (Dividends / Stock Splits columns may appear)
hist_actions = t_equity.history(period="1y", interval="1d", auto_adjust=True, actions=True)
hist_actions.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-12-15 00:00:00-05:00,280.149994,280.149994,272.839996,274.109985,50409100,0.0,0.0
2025-12-16 00:00:00-05:00,272.820007,275.500000,271.790009,274.609985,37648600,0.0,0.0
2025-12-17 00:00:00-05:00,275.010010,276.160004,271.640015,271.839996,50138700,0.0,0.0
2025-12-18 00:00:00-05:00,273.609985,273.630005,266.950012,272.190002,51630700,0.0,0.0
2025-12-19 00:00:00-05:00,272.149994,274.600006,269.899994,273.670013,144599200,0.0,0.0


In [32]:
# History with embedded actions (Dividends / Stock Splits columns may appear)
hist_actions = t_equity.history(period="1y", interval="1d", auto_adjust=False, actions=True)
hist_actions.tail()

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2025-12-15 00:00:00-05:00,280.149994,280.149994,272.839996,274.109985,274.109985,50409100,0.0,0.0
2025-12-16 00:00:00-05:00,272.820007,275.500000,271.790009,274.609985,274.609985,37648600,0.0,0.0
2025-12-17 00:00:00-05:00,275.010010,276.160004,271.640015,271.839996,271.839996,50138700,0.0,0.0
2025-12-18 00:00:00-05:00,273.609985,273.630005,266.950012,272.190002,272.190002,51630700,0.0,0.0
2025-12-19 00:00:00-05:00,272.149994,274.600006,269.899994,273.670013,273.670013,144599200,0.0,0.0


In [36]:
# Bulk download for multiple tickers
bulk = yf.download([EQUITY, "MSFT"], period="6mo", interval="1d", actions=True, auto_adjust=False, threads=True)
bulk.tail()

[*********************100%***********************]  2 of 2 completed


Price        Adj Close                   Close             Dividends       \
Ticker            AAPL        MSFT        AAPL        MSFT      AAPL MSFT   
Date                                                                        
2025-12-15  274.109985  474.820007  274.109985  474.820007       0.0  0.0   
2025-12-16  274.609985  476.390015  274.609985  476.390015       0.0  0.0   
2025-12-17  271.839996  476.119995  271.839996  476.119995       0.0  0.0   
2025-12-18  272.190002  483.980011  272.190002  483.980011       0.0  0.0   
2025-12-19  273.670013  485.920013  273.670013  485.920013       0.0  0.0   

Price             High                     Low                    Open  \
Ticker            AAPL        MSFT        AAPL        MSFT        AAPL   
Date                                                                     
2025-12-15  280.149994  480.720001  272.839996  472.519989  280.149994   
2025-12-16  275.500000  477.890015  271.790009  470.880005  272.820007   
2025-12-17  276.160004  480.000000  271.640015  475.000000  275.010010   
2025-12-18  273.630005  489.600006  266.950012  477.890015  273.609985   
2025-12-19  274.600006  487.850006  269.899994  482.489990  272.149994   

Price                  Stock Splits          Volume            
Ticker            MSFT         AAPL MSFT       AAPL      MSFT  
Date                                                           
2025-12-15  480.100006          0.0  0.0   50409100  23727700  
2025-12-16  471.910004          0.0  0.0   37648600  20705600  
2025-12-17  476.910004          0.0  0.0   50138700  24527200  
2025-12-18  478.190002          0.0  0.0   51630700  28573500  
2025-12-19  487.359985          0.0  0.0  144599200  70824900

## 4) Corporate actions (dividends, splits, capital gains, combined actions)

In [37]:
dividends = t_equity.dividends
splits = t_equity.splits

try:
    cap_gains = t_equity.capital_gains
except Exception as e:
    cap_gains = f"Not available / error: {e!r}"

actions = t_equity.actions

(dividends.tail(), splits.tail(), str(cap_gains)[:120])

(Date
 2024-11-08 00:00:00-05:00    0.25
 2025-02-10 00:00:00-05:00    0.25
 2025-05-12 00:00:00-04:00    0.26
 2025-08-11 00:00:00-04:00    0.26
 2025-11-10 00:00:00-05:00    0.26
 Name: Dividends, dtype: float64,
 Date
 1987-06-16 00:00:00-04:00    2.0
 2000-06-21 00:00:00-04:00    2.0
 2005-02-28 00:00:00-05:00    2.0
 2014-06-09 00:00:00-04:00    7.0
 2020-08-31 00:00:00-04:00    4.0
 Name: Stock Splits, dtype: float64,
 'Series([], dtype: object)')

In [38]:
actions.tail()

,Dividends,Stock Splits
Date,,
2024-11-08 00:00:00-05:00,0.25,0.0
2025-02-10 00:00:00-05:00,0.25,0.0
2025-05-12 00:00:00-04:00,0.26,0.0
2025-08-11 00:00:00-04:00,0.26,0.0
2025-11-10 00:00:00-05:00,0.26,0.0


## 5) Snapshot metrics: info (large dict) and fast_info (lightweight)

In [39]:
info_ok, info, info_err = safe_getattr(t_equity, "info")
fast_ok, fast, fast_err = safe_getattr(t_equity, "fast_info")

print("info:", info_ok, "-" if info_ok else info_err)
print("fast_info:", fast_ok, "-" if fast_ok else fast_err)

# If info is present, show a small subset of commonly useful keys (when available)
if info_ok and isinstance(info, dict):
    keys = [
        "longName","symbol","sector","industry","country",
        "marketCap","enterpriseValue",
        "trailingPE","forwardPE","priceToBook","beta",
        "dividendYield","profitMargins","grossMargins","operatingMargins",
        "returnOnEquity","returnOnAssets",
        "totalRevenue","netIncomeToCommon"
    ]
    subset = {k: info.get(k, None) for k in keys}
    display(pd.Series(subset))

info: True -
fast_info: True -


longName                       Apple Inc.
symbol                               AAPL
sector                         Technology
industry             Consumer Electronics
country                     United States
marketCap                   4061369729024
enterpriseValue             4101524422656
trailingPE                       36.63588
forwardPE                       30.003498
priceToBook                       54.8327
beta                                1.107
dividendYield                        0.38
profitMargins                     0.26915
grossMargins                      0.46905
operatingMargins                  0.31647
returnOnEquity                    1.71422
returnOnAssets                    0.22964
totalRevenue                 416161005568
netIncomeToCommon            112010002432
dtype: object

In [40]:
# Normalize the full info dict to a table (can be large)
if info_ok and isinstance(info, dict):
    info_series = pd.Series(info).sort_index()
    display(info_series.head(40))

52WeekChange                                                                                                                                                      0.07208
SandP52WeekChange                                                                                                                                                0.144027
address1                                                                                                                                               One Apple Park Way
allTimeHigh                                                                                                                                                        288.62
allTimeLow                                                                                                                                                       0.049107
ask                                                                                                                                                   

## 6) Calendar, SEC filings, News

In [41]:
cal_ok, cal, cal_err = safe_getattr(t_equity, "calendar")
sec_ok, sec, sec_err = safe_getattr(t_equity, "sec_filings")
news_ok, news, news_err = safe_getattr(t_equity, "news")

print("calendar:", summarize_value(cal) if cal_ok else cal_err)
print("sec_filings:", summarize_value(sec) if sec_ok else sec_err)
print("news:", summarize_value(news) if news_ok else news_err)

cal if cal_ok else None

calendar: dict keys(n=9): ['Dividend Date', 'Ex-Dividend Date', 'Earnings Date', 'Earnings High', 'Earnings Low', 'Earnings Average', 'Revenue High', 'Revenue Low', 'Revenue Average']
sec_filings: list len=68
news: list len=10


{'Dividend Date': datetime.date(2025, 11, 12),
 'Ex-Dividend Date': datetime.date(2025, 11, 9),
 'Earnings Date': [datetime.date(2026, 1, 29)],
 'Earnings High': 2.76,
 'Earnings Low': 2.51,
 'Earnings Average': 2.6647,
 'Revenue High': 142741000000,
 'Revenue Low': 136679500000,
 'Revenue Average': 138253076640}

In [42]:
if news_ok and isinstance(news, list) and len(news) > 0:
    display(pd.DataFrame(news).head(8))

,id,content
0,0e883192-60e2-4505-a7f5-3e1dd84e8983,"{'id': '0e883192-60e2-4505-a7f5-3e1dd84e8983', 'contentType': 'STORY', 'title': ''We've all grown up with radiators - underfloor heating..."
1,29da4429-e231-360e-837f-be9ebb1a2f49,"{'id': '29da4429-e231-360e-837f-be9ebb1a2f49', 'contentType': 'STORY', 'title': 'Best Stock to Buy Right Now: Apple vs. Amazon', 'descri..."
2,8fe6095b-467c-3bb1-97ff-0748cd9c748b,"{'id': '8fe6095b-467c-3bb1-97ff-0748cd9c748b', 'contentType': 'STORY', 'title': 'OpenAI vs. Apple? Sam Altman is setting his sights on w..."
3,eb38dc58-7c08-384c-bf53-d95527a704c3,"{'id': 'eb38dc58-7c08-384c-bf53-d95527a704c3', 'contentType': 'STORY', 'title': 'Analysts Have a Moderate Buy Rating on Apple Inc. (AAPL..."
4,aa3cecef-6c58-481b-9447-ac82d03c8fe4,"{'id': 'aa3cecef-6c58-481b-9447-ac82d03c8fe4', 'contentType': 'STORY', 'title': ''Childminding is an exciting, AI-proof way to make a go..."
5,8c220dd6-74d5-3970-9b87-7fcf28ccabfe,"{'id': '8c220dd6-74d5-3970-9b87-7fcf28ccabfe', 'contentType': 'STORY', 'title': '5 predictions for tech and AI in 2026 from mega-bull Da..."
6,83faeafe-1349-355b-ac46-f730bbf4bc5f,"{'id': '83faeafe-1349-355b-ac46-f730bbf4bc5f', 'contentType': 'STORY', 'title': '‘I don’t know how much my wife earns’: I’m 63 with $6.4..."
7,a6c02fd5-7b13-33fa-878c-4ba88f6f2f3d,"{'id': 'a6c02fd5-7b13-33fa-878c-4ba88f6f2f3d', 'contentType': 'VIDEO', 'title': 'Nvidia, Tesla dominate Dec. trading as 2025 options dem..."


## 7) Holders and insiders

In [43]:
holders_fields = [
    "major_holders", "institutional_holders", "mutualfund_holders",
    "insider_purchases", "insider_transactions", "insider_roster_holders"
]

for f in holders_fields:
    ok, v, err = safe_getattr(t_equity, f)
    print(f"{f:>22} | ok={ok} | {summarize_value(v) if ok else err}")

         major_holders | ok=True | DataFrame shape=(4, 1), cols=['Value']
 institutional_holders | ok=True | DataFrame shape=(10, 6), cols=['Date Reported', 'Holder', 'pctHeld', 'Shares', 'Value', 'pctChange']
    mutualfund_holders | ok=True | DataFrame shape=(10, 6), cols=['Date Reported', 'Holder', 'pctHeld', 'Shares', 'Value', 'pctChange']
     insider_purchases | ok=True | DataFrame shape=(7, 3), cols=['Insider Purchases Last 6m', 'Shares', 'Trans']
  insider_transactions | ok=True | DataFrame shape=(76, 9), cols=['Shares', 'Value', 'URL', 'Text', 'Insider', 'Position', 'Transaction', 'Start Date', 'Ownership']
insider_roster_holders | ok=True | DataFrame shape=(10, 7), cols=['Name', 'Position', 'URL', 'Most Recent Transaction', 'Latest Transaction Date', 'Shares Owned Directly', 'Position Direct Date']


In [44]:
# Example: institutional holders (if available)
ok, inst, err = safe_getattr(t_equity, "institutional_holders")
inst.head() if ok and isinstance(inst, pd.DataFrame) and not inst.empty else inst

,Date Reported,Holder,pctHeld,Shares,Value,pctChange
0,2025-09-30,Vanguard Group Inc,0.0947,1399427162,382981250215,-0.0117
1,2025-09-30,Blackrock Inc.,0.0776,1146332274,313716768818,-0.0022
2,2025-09-30,State Street Corporation,0.0404,597501113,163518137617,-0.0062
3,2025-09-30,JPMORGAN CHASE & CO,0.0320,473311062,129531044693,1.2055
4,2025-09-30,"Geode Capital Management, LLC",0.0241,356166414,97472067301,0.0040


## 8) Earnings (history + dates)

In [45]:
earn_fields = ["earnings", "quarterly_earnings", "earnings_dates", "earnings_history"]
for f in earn_fields:
    ok, v, err = safe_getattr(t_equity, f)
    print(f"{f:>18} | ok={ok} | {summarize_value(v) if ok else err}")

c:\Users\herie\miniconda3\envs\a\Lib\site-packages\yfinance\scrapers\fundamentals.py:36: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)


          earnings | ok=True | None
quarterly_earnings | ok=True | None
    earnings_dates | ok=True | DataFrame shape=(25, 3), cols=['EPS Estimate', 'Reported EPS', 'Surprise(%)']
  earnings_history | ok=True | DataFrame shape=(4, 4), cols=['epsActual', 'epsEstimate', 'epsDifference', 'surprisePercent']


In [46]:
# Show earnings dates (often useful for event studies)
ok, ed, err = safe_getattr(t_equity, "earnings_dates")
ed.head() if ok and isinstance(ed, pd.DataFrame) else ed

,EPS Estimate,Reported EPS,Surprise(%)
Earnings Date,,,
2026-01-29 16:00:00-05:00,2.67,NaN,NaN
2025-10-30 16:00:00-04:00,1.77,1.85,4.47
2025-07-31 16:00:00-04:00,1.43,1.57,9.48
2025-05-01 16:00:00-04:00,1.63,1.65,1.50
2025-01-30 16:00:00-05:00,2.35,2.40,2.26


## 9) Financial statements (Income Statement, Balance Sheet, Cash Flow)

In [47]:
stmt_fields = [
    "income_stmt", "quarterly_income_stmt", "ttm_income_stmt",
    "balance_sheet", "quarterly_balance_sheet",
    "cash_flow", "quarterly_cash_flow", "ttm_cash_flow"
]
for f in stmt_fields:
    ok, v, err = safe_getattr(t_equity, f)
    print(f"{f:>24} | ok={ok} | {summarize_value(v) if ok else err}")

             income_stmt | ok=True | DataFrame shape=(39, 5), cols=[Timestamp('2025-09-30 00:00:00'), Timestamp('2024-09-30 00:00:00'), Timestamp('2023-09-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2021-09-30 00:00:00')]
   quarterly_income_stmt | ok=True | DataFrame shape=(33, 5), cols=[Timestamp('2025-09-30 00:00:00'), Timestamp('2025-06-30 00:00:00'), Timestamp('2025-03-31 00:00:00'), Timestamp('2024-12-31 00:00:00'), Timestamp('2024-09-30 00:00:00')]
         ttm_income_stmt | ok=True | DataFrame shape=(33, 1), cols=[Timestamp('2025-09-30 00:00:00')]
           balance_sheet | ok=True | DataFrame shape=(69, 5), cols=[Timestamp('2025-09-30 00:00:00'), Timestamp('2024-09-30 00:00:00'), Timestamp('2023-09-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2021-09-30 00:00:00')]
 quarterly_balance_sheet | ok=True | DataFrame shape=(65, 5), cols=[Timestamp('2025-09-30 00:00:00'), Timestamp('2025-06-30 00:00:00'), Timestamp('2025-03-31 00:00:00'), Timestamp('202

In [48]:
# Example: income statement
ok, is_df, err = safe_getattr(t_equity, "income_stmt")
is_df.head(18) if ok and isinstance(is_df, pd.DataFrame) else is_df

,2025-09-30,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Tax Effect Of Unusual Items,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
Tax Rate For Calcs,1.560000e-01,2.410000e-01,1.470000e-01,1.620000e-01,NaN
Normalized EBITDA,1.447480e+11,1.346610e+11,1.258200e+11,1.305410e+11,NaN
Net Income From Continuing Operation Net Minority Interest,1.120100e+11,9.373600e+10,9.699500e+10,9.980300e+10,NaN
Reconciled Depreciation,1.169800e+10,1.144500e+10,1.151900e+10,1.110400e+10,NaN
Reconciled Cost Of Revenue,2.209600e+11,2.103520e+11,2.141370e+11,2.235460e+11,NaN
EBITDA,1.447480e+11,1.346610e+11,1.258200e+11,1.305410e+11,NaN
EBIT,1.330500e+11,1.232160e+11,1.143010e+11,1.194370e+11,NaN
Net Interest Income,NaN,NaN,-1.830000e+08,-1.060000e+08,1.980000e+08
Interest Expense,NaN,NaN,3.933000e+09,2.931000e+09,2.645000e+09


In [49]:
# Example: cash flow
ok, cf_df, err = safe_getattr(t_equity, "cash_flow")
cf_df.head(18) if ok and isinstance(cf_df, pd.DataFrame) else cf_df

,2025-09-30,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Free Cash Flow,9.876700e+10,1.088070e+11,9.958400e+10,1.114430e+11,NaN
Repurchase Of Capital Stock,-9.071100e+10,-9.494900e+10,-7.755000e+10,-8.940200e+10,NaN
Repayment Of Debt,-1.093200e+10,-9.958000e+09,-1.115100e+10,-9.543000e+09,NaN
Issuance Of Debt,4.481000e+09,0.000000e+00,5.228000e+09,5.465000e+09,NaN
Issuance Of Capital Stock,NaN,NaN,NaN,NaN,1.105000e+09
Capital Expenditure,-1.271500e+10,-9.447000e+09,-1.095900e+10,-1.070800e+10,NaN
Interest Paid Supplemental Data,NaN,NaN,3.803000e+09,2.865000e+09,2.687000e+09
Income Tax Paid Supplemental Data,4.336900e+10,2.610200e+10,1.867900e+10,1.957300e+10,NaN
End Cash Position,3.593400e+10,2.994300e+10,3.073700e+10,2.497700e+10,NaN
Beginning Cash Position,2.994300e+10,3.073700e+10,2.497700e+10,3.592900e+10,NaN


## 10) Analyst recommendations, upgrades/downgrades, price targets

In [50]:
analyst_fields = ["recommendations", "recommendations_summary", "upgrades_downgrades", "analyst_price_targets"]
for f in analyst_fields:
    ok, v, err = safe_getattr(t_equity, f)
    print(f"{f:>26} | ok={ok} | {summarize_value(v) if ok else err}")

           recommendations | ok=True | DataFrame shape=(4, 6), cols=['period', 'strongBuy', 'buy', 'hold', 'sell', 'strongSell']
   recommendations_summary | ok=True | DataFrame shape=(4, 6), cols=['period', 'strongBuy', 'buy', 'hold', 'sell', 'strongSell']
       upgrades_downgrades | ok=True | DataFrame shape=(968, 7), cols=['Firm', 'ToGrade', 'FromGrade', 'Action', 'priceTargetAction', 'currentPriceTarget', 'priorPriceTarget']
     analyst_price_targets | ok=True | dict keys(n=5): ['current', 'high', 'low', 'mean', 'median']


In [51]:
# Show a sample of recommendations (if available)
ok, rec, err = safe_getattr(t_equity, "recommendations")
rec.tail() if ok and isinstance(rec, pd.DataFrame) else rec

,period,strongBuy,buy,hold,sell,strongSell
0,0m,5,24,15,1,3
1,-1m,5,24,15,1,3
2,-2m,5,24,15,1,3
3,-3m,5,23,15,1,3


## 11) Estimates and trends (EPS trend/revisions, revenue/earnings estimates, growth estimates)

In [52]:
estimate_fields = [
    "earnings_estimate", "revenue_estimate",
    "eps_trend", "eps_revisions", "growth_estimates"
]
for f in estimate_fields:
    ok, v, err = safe_getattr(t_equity, f)
    print(f"{f:>18} | ok={ok} | {summarize_value(v) if ok else err}")

 earnings_estimate | ok=True | DataFrame shape=(4, 6), cols=['avg', 'low', 'high', 'yearAgoEps', 'numberOfAnalysts', 'growth']
  revenue_estimate | ok=True | DataFrame shape=(4, 6), cols=['avg', 'low', 'high', 'numberOfAnalysts', 'yearAgoRevenue', 'growth']
         eps_trend | ok=True | DataFrame shape=(4, 5), cols=['current', '7daysAgo', '30daysAgo', '60daysAgo', '90daysAgo']
     eps_revisions | ok=True | DataFrame shape=(4, 4), cols=['upLast7days', 'upLast30days', 'downLast30days', 'downLast7Days']
  growth_estimates | ok=True | DataFrame shape=(5, 2), cols=['stockTrend', 'indexTrend']


In [53]:
# Example: EPS trend
ok, eps_tr, err = safe_getattr(t_equity, "eps_trend")
eps_tr if ok else err

,current,7daysAgo,30daysAgo,60daysAgo,90daysAgo
period,,,,,
0q,2.66574,2.66367,2.66153,2.48232,2.49200
+1q,1.83115,1.83115,1.83227,1.80014,1.80449
0y,8.24606,8.24657,8.23661,7.98373,7.99666
+1y,9.10187,9.09382,9.07626,8.80731,8.83763


## 12) ESG / Sustainability

In [54]:
ok, esg, err = safe_getattr(t_equity, "sustainability")
print("sustainability:", summarize_value(esg) if ok else err)
esg if ok else None

sustainability: DataFrame shape=(0, 0), cols=[]


""


## 13) Options: expirations + option chain tables

In [55]:
ok, expiries, err = safe_getattr(t_equity, "options")
print("expirations:", (list(expiries)[:5] if ok else err))

if ok and isinstance(expiries, tuple) and len(expiries) > 0:
    first_exp = expiries[0]
    chain = t_equity.option_chain(first_exp)
    display(chain.calls.head())
    display(chain.puts.head())
    print("Underlying keys (sample):", list(chain.underlying.keys())[:20] if isinstance(chain.underlying, dict) else chain.underlying)

expirations: ['2025-12-26', '2026-01-02', '2026-01-09', '2026-01-16', '2026-01-23']


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL251226C00110000,2025-12-05 18:39:45+00:00,110.0,170.30,162.25,165.15,0.00,0.000000,1.0,1,2.453129,True,REGULAR,USD
1,AAPL251226C00120000,2025-12-08 15:37:49+00:00,120.0,158.22,152.35,155.15,0.00,0.000000,1.0,1,2.460941,True,REGULAR,USD
2,AAPL251226C00125000,2025-12-03 19:20:43+00:00,125.0,160.40,147.35,150.25,0.00,0.000000,NaN,4,2.480473,True,REGULAR,USD
3,AAPL251226C00145000,2025-12-11 19:37:24+00:00,145.0,132.91,127.35,130.20,0.00,0.000000,NaN,1,1.992188,True,REGULAR,USD
4,AAPL251226C00150000,2025-12-19 20:12:09+00:00,150.0,121.10,122.30,125.35,-2.18,-1.768333,1.0,10,1.984375,True,REGULAR,USD


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL251226P00120000,2025-12-19 18:09:39+00:00,120.0,0.01,0.0,0.36,-0.01,-50.0,1.0,1.0,2.703128,False,REGULAR,USD
1,AAPL251226P00125000,2025-12-15 16:09:11+00:00,125.0,0.03,NaN,0.37,0.00,0.0,NaN,1.0,2.837894,False,REGULAR,USD
2,AAPL251226P00130000,2025-11-21 15:08:52+00:00,130.0,0.02,0.0,0.38,0.00,0.0,8.0,44.0,2.476566,False,REGULAR,USD
3,AAPL251226P00145000,2025-11-20 20:51:13+00:00,145.0,0.04,0.0,0.40,0.00,0.0,199.0,202.0,2.154301,False,REGULAR,USD
4,AAPL251226P00150000,2025-11-13 20:34:27+00:00,150.0,0.02,0.0,0.41,0.00,0.0,NaN,10.0,2.054692,False,REGULAR,USD


Underlying keys (sample): ['language', 'region', 'quoteType', 'typeDisp', 'quoteSourceName', 'triggerable', 'customPriceAlertConfidence', 'currency', 'shortName', 'longName', 'marketState', 'fiftyDayAverage', 'fiftyDayAverageChange', 'fiftyDayAverageChangePercent', 'twoHundredDayAverage', 'twoHundredDayAverageChange', 'twoHundredDayAverageChangePercent', 'marketCap', 'forwardPE', 'priceToBook']


## 14) Funds/ETF-specific data (FundsData)

In [56]:
ok, fd, err = safe_getattr(t_etf, "funds_data")
print("funds_data:", type(fd).__name__ if ok else err)

# FundsData is an object; its attributes vary by yfinance version.
if ok and fd is not None:
    attrs = [a for a in dir(fd) if not a.startswith("_")]
    print("FundsData public attrs (sample):", attrs[:50])

funds_data: FundsData
FundsData public attrs (sample): ['asset_classes', 'bond_holdings', 'bond_ratings', 'description', 'equity_holdings', 'fund_operations', 'fund_overview', 'quote_type', 'sector_weightings', 'top_holdings']


## 15) Full capability probe (for a given symbol)

In [57]:
FIELDS = [
    "isin",
    "major_holders","institutional_holders","mutualfund_holders",
    "insider_purchases","insider_transactions","insider_roster_holders",
    "dividends","capital_gains","splits","actions",
    "shares",
    "info","fast_info","calendar","sec_filings",
    "recommendations","recommendations_summary","upgrades_downgrades",
    "earnings","quarterly_earnings",
    "income_stmt","quarterly_income_stmt","ttm_income_stmt",
    "balance_sheet","quarterly_balance_sheet",
    "cash_flow","quarterly_cash_flow","ttm_cash_flow",
    "analyst_price_targets",
    "earnings_estimate","revenue_estimate","earnings_history",
    "eps_trend","eps_revisions","growth_estimates",
    "sustainability",
    "options",
    "news",
    "earnings_dates",
    "history_metadata",
    "funds_data",
]

probe = probe_ticker(EQUITY, FIELDS)
probe

c:\Users\herie\miniconda3\envs\a\Lib\site-packages\yfinance\scrapers\fundamentals.py:36: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: AAPL"}}}


,field,ok,type,summary,error
0,actions,True,DataFrame,"DataFrame shape=(94, 2), cols=['Dividends', 'Stock Splits']",None
1,analyst_price_targets,True,dict,"dict keys(n=5): ['current', 'high', 'low', 'mean', 'median']",None
2,balance_sheet,True,DataFrame,"DataFrame shape=(69, 5), cols=[Timestamp('2025-09-30 00:00:00'), Timestamp('2024-09-30 00:00:00'), Timestamp('2023-09-30 00:00:00'), Tim...",None
3,calendar,True,dict,"dict keys(n=9): ['Dividend Date', 'Ex-Dividend Date', 'Earnings Date', 'Earnings High', 'Earnings Low', 'Earnings Average', 'Revenue Hig...",None
4,capital_gains,True,Series,"Series len=0, name=None, dtype=object",None
5,cash_flow,True,DataFrame,"DataFrame shape=(53, 5), cols=[Timestamp('2025-09-30 00:00:00'), Timestamp('2024-09-30 00:00:00'), Timestamp('2023-09-30 00:00:00'), Tim...",None
6,dividends,True,Series,"Series len=89, name=Dividends, dtype=float64",None
7,earnings,True,NoneType,None,None
8,earnings_dates,True,DataFrame,"DataFrame shape=(25, 3), cols=['EPS Estimate', 'Reported EPS', 'Surprise(%)']",None
9,earnings_estimate,True,DataFrame,"DataFrame shape=(4, 6), cols=['avg', 'low', 'high', 'yearAgoEps', 'numberOfAnalysts', 'growth']",None
